In [1]:
import selenium
from selenium import webdriver
import requests
from bs4 import BeautifulSoup
import urllib.request

import time
import io
import os
import hashlib

from PIL import Image

import pandas as pd

In [2]:
#import selenium
# This is the path I use
# DRIVER_PATH = '.../Desktop/Scraping/chromedriver 2'
# Put the path for your ChromeDriver here

# https://selenium-python.readthedocs.io/api.html

DRIVER_PATH = './chromedriver'
wd = webdriver.Chrome(executable_path=DRIVER_PATH)

In [3]:
wd.get('https://google.com')

In [4]:
#search_box = wd.find_element_by_css_selector('input.gLFyf')
#search_box.send_keys('apple')

In [5]:
#wd.quit()

In [6]:
def fetch_image_urls(query:str, max_links_to_fetch:int, wd:webdriver, sleep_between_interactions:int=1):
    
    def scroll_to_end(wd):
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        #https://www.datacamp.com/community/tutorials/python-time-sleep?utm_source=adwords_ppc&utm_campaignid=898687156&utm_adgroupid=48947256715&utm_device=c&utm_keyword=&utm_matchtype=b&utm_network=g&utm_adpostion=&utm_creative=332602034349&utm_targetid=aud-299261629574:dsa-429603003980&utm_loc_interest_ms=&utm_loc_physical_ms=9061132&gclid=CjwKCAjw4rf6BRAvEiwAn2Q76qBrQ96sqtFvsYYyeGSj3wgtw_gtUr9Eq6OrWO8IDunXoL4d3VlImxoCgJoQAvD_BwE
        
        time.sleep(sleep_between_interactions)    
    
    # build the google query
    search_url = "https://www.google.com/search?safe=off&site=&tbm=isch&source=hp&q={q}&oq={q}&gs_l=img"

    # load the page
    wd.get(search_url.format(q=query))

    image_urls = set()
    image_count = 0
    results_start = 0
    while image_count < max_links_to_fetch:
        scroll_to_end(wd)

        # get all image thumbnail results by finding the corresponding css element
        #https://selenium-python.readthedocs.io/locating-elements.html
        
        thumbnail_results = wd.find_elements_by_css_selector("img.Q4LuWd")
        number_results = len(thumbnail_results)
        
        print(f"Found: {number_results} search results. Extracting links from {results_start}:{number_results}")
        
        for img in thumbnail_results[results_start:number_results]:
            # try to click every thumbnail so that we can get the real image behind it
            try:
                img.click()
                time.sleep(sleep_between_interactions)
            except Exception:
                continue

            # extract image urls    
            actual_images = wd.find_elements_by_css_selector('img.n3VNCb')
            for actual_image in actual_images:
                #https://www.geeksforgeeks.org/get_attribute-element-method-selenium-python/
                
                if actual_image.get_attribute('src') and 'http' in actual_image.get_attribute('src'):
                    image_urls.add(actual_image.get_attribute('src'))

            image_count = len(image_urls)

            if len(image_urls) >= max_links_to_fetch:
                print(f"Found: {len(image_urls)} image links, done!")
                break
        else:
            print("Found:", len(image_urls), "image links, looking for more ...")
            time.sleep(30)
            return
            load_more_button = wd.find_element_by_css_selector(".mye4qd")
            if load_more_button:
                #https://stackoverflow.com/questions/12792236/sent-user-to-page-bottom-by-javascript
                wd.execute_script("document.querySelector('.mye4qd').click();")

        # move the result startpoint further down
        results_start = len(thumbnail_results)

    return image_urls

In [7]:
#def persist_image(folder_path:str, url:str):
#    try:
#        image_content = requests.get(url).content

#    except Exception as e:
#        print(f"ERROR - Could not download {url} - {e}")

#    try:
#        image_file = io.BytesIO(image_content)
#        image = Image.open(image_file)#.convert('RGB')
#        file_path = os.path.join(folder_path, hashlib.sha1(image_content).hexdigest()[:10] + '.jpg')
#        with open(file_path, 'wb') as f:
#            image.save(f, "JPEG", quality=85)
#        print(f"SUCCESS - saved {url} - as {file_path}")
#    except Exception as e:
#        print(f"ERROR - Could not save {url} - {e}")

In [8]:
def save_image(url, file_name):
    
    #https://stackoverflow.com/questions/8286352/how-to-save-an-image-locally-using-python-whose-url-address-i-already-know
    
    # adding information about user agent
    opener=urllib.request.build_opener()
    opener.addheaders=[('User-Agent','Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1941.0 Safari/537.36')]
    urllib.request.install_opener(opener)
    
    # calling urlretrieve function to get resource
    urllib.request.urlretrieve(url, file_name)

In [9]:
def search_and_download(search_term:str, driver_path:str, target_path='./images_new',number_images=5):
    #https://towardsdatascience.com/how-to-download-an-image-using-python-38a75cfa21c
    #https://gist.github.com/wenbert/758257
    
    
    target_folder = os.path.join(target_path,'_'.join(search_term.lower().split(' ')))

    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    #with webdriver.Chrome(executable_path=driver_path) as wd:
    #    res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
    
    i = 1    
    for elem in res:
        with webdriver.Chrome(executable_path=driver_path) as wd:
            res = fetch_image_urls(search_term, number_images, wd=wd, sleep_between_interactions=0.5)
        
        file_name = os.path.join(target_folder, f'{search_term.replace(" ", "_")}_{i}.jpg')
        save_image(elem, file_name)
        i += 1

In [10]:
search_term_df = pd.read_csv('./50_food_classes.txt', header=None)
#search_term_df

In [11]:
#search_and_download(
#        search_term = 'chocolate',
#        driver_path = DRIVER_PATH,
#        number_images = 10
#    )

In [12]:
for i in search_term_df[0]:
    if i == 'apple':
        search_term = 'single apple fruit'
    else:
        search_term = i
    
    search_and_download(
        search_term = search_term,
        driver_path = DRIVER_PATH,
        number_images = 50
    )

Found: 100 search results. Extracting links from 0:100
Found: 50 image links, done!
Found: 100 search results. Extracting links from 0:100
Found: 51 image links, done!
Found: 100 search results. Extracting links from 0:100
Found: 51 image links, done!


TimeoutError: [Errno 60] Operation timed out

In [ ]:
wd.quit()

# General links

https://simply-python.com/2015/05/18/saving-images-from-google-search-using-selenium-and-python/

https://towardsdatascience.com/image-scraping-with-python-a96feda8af2d

https://towardsdatascience.com/web-scraping-with-python-a-to-copy-z-277a445d64c7#:~:text=Scraping%20Scraping%20Scraping&text=In%20order%20to%20inspect%20the,%E2%86%92%20hit%20%E2%80%9CInspect%20element%E2%80%9D.&text=Then%2C%20locate%20the%20data%20you,scrape%20and%20click%20on%20it.&text=The%20CSS%20class%20of%20the,the%20data%20from%20the%20html.

https://towardsdatascience.com/web-scraping-5649074f3ead

https://towardsdatascience.com/data-science-skills-web-scraping-using-python-d1a85ef607ed

https://towardsdatascience.com/web-scraping-is-now-legal-6bf0e5730a78

https://towardsdatascience.com/web-scraping-basics-selenium-and-beautiful-soup-applied-to-searching-for-campsite-availability-4a8de1decac9

https://gist.github.com/genekogan/ebd77196e4bf0705db51f86431099e57

https://google-images-download.readthedocs.io/en/latest/

https://medium.com/dreamcatcher-its-blog/5-simple-tips-for-improving-automated-web-testing-or-efficient-web-crawling-using-selenium-python-43038d7b7916

Selenium documentation - https://selenium-python.readthedocs.io/api.html

Selenium documentation - https://selenium-python.readthedocs.io/getting-started.html

Requests - https://requests.readthedocs.io/en/master/user/quickstart/